# Домашнее задание 3 [10 баллов]


# До 27.03.18 23:59

Рассмотрим задачу бинарной классификации. Пусть дано два списка имен: мужские и женские имена. Требуется разработать классификатор, который по данному имени будет определять мужское оно или женское.

Данные: 
* Женские имена: female.txt
* Мужские имена: male.txt

## Часть 1. Предварительная

### Предварительная обработка данных [2 балла]

1. Удалите неоднозначные имена (те имена, которые являются и мужскими, и женскими дновременно), если такие есть; 
2. Создайте обучающее и тестовое множество так, чтобы в обучающем множестве классы были сбалансированы, т.е. к классу принадлежало бы одинаковое количество имен;

In [1]:
# прочитаем файлы
female = open('female.txt', 'r').read().split('\n')
male = open('male.txt', 'r').read().split('\n')
print(len(female), len(male))

5001 2943


#### Удаляем неоднозначные имена

In [2]:
# найдем имена, которые одинаковы для двух списков
result = set(female) & set(male)

In [3]:
# удалим их!
female = list(set(female) - result)
male = list(set(male) - result)

In [4]:
print(len(female), len(male))

4636 2578


In [5]:
print(male[:100])

['Pryce', 'Michail', 'Tomlin', 'Piet', 'Everett', 'Isidore', 'Marven', 'Jim', 'Farley', 'Orlando', 'Ibrahim', 'Waring', 'Cortese', 'Jakob', 'Greggory', 'Cheston', 'Raymond', 'Mauricio', 'Torrance', 'Arne', 'Buck', 'Victor', 'Georges', 'Rufus', 'Stearn', 'Archon', 'Ignacius', 'Merwin', 'Zach', 'Taddeus', 'Zebulon', 'Leonard', 'Clinton', 'Artie', 'Harry', 'Hale', 'Jefferson', 'Joseph', 'Timotheus', 'Cy', 'Sturgis', 'Giffy', 'Arvie', 'Alexei', 'Denis', 'Abdul', 'Town', 'Skipton', 'Donovan', 'Hayes', 'Yuri', 'Theodore', 'Vance', 'Merril', 'Walton', 'Hewet', 'Hogan', 'Dexter', 'Warren', 'Dustin', 'Bailie', 'Hermon', 'Dyson', 'Hal', 'Bartolomei', 'Sheff', 'Easton', 'Gregg', 'Sol', 'Brice', 'Irving', 'Linoel', 'Julio', 'Ferdie', 'Allan', 'Sivert', 'Syd', 'Virge', 'Darren', 'Lonny', 'Ev', 'Horace', 'Skipper', 'Bennett', 'Lon', 'Anatoly', 'Walsh', 'Pietro', 'Frederic', 'Bogdan', 'Giorgio', 'Marcel', 'Myles', 'Nelson', 'Murphy', 'Wilfrid', 'Tremain', 'Darrick', 'Istvan', 'Yancy']


#### Балансируем выборки

In [6]:
# oversampling!
missing = len(female) - len(male)
male += male[:missing]

In [7]:
# suffling
import random
random.shuffle(male)

In [8]:
print(len(female), len(male))

4636 4636


In [15]:
boundary = int(len(female) * 0.8)
x_train = female[:boundary] + male[:boundary]
x_test = female[boundary:] + male[boundary:]
y_train = ['female'] * (boundary ) + ['male'] * (boundary)
y_test = ['female'] * (len(x_test)//2) + ['male'] * (len(x_test)//2)
print(len(x_train), len(x_test), len(y_train), len(y_test))

7416 1856 7416 1856


In [16]:
import pandas as pd
index = random.sample(range(len(x_train)), len(x_train))
df = pd.DataFrame({'name': x_train, 'class': y_train})
df.head()

,class,name
0,female,Astrix
1,female,Sylvia
2,female,Veda
3,female,Tammara
4,female,Tonya


In [18]:
df = df.sample(frac=1).reset_index(drop=True)
df.head()

,class,name
0,female,Jackelyn
1,female,Beth
2,male,Milo
3,male,Harvey
4,female,Elyse


##  Часть 2. Базовая 

### Базовый метод классификации [3 балла]

Используйте метод наивного Байеса или логистическую регрессию для классификации имен: в качестве признаков используйте символьные $n$-граммы. Сравните результаты, получаемые при разных $n=2,3,4$ по $F$-мере и аккуратности. В каких случаях метод ошибается?

Для генерации $n$-грамм используйте:

In [1]:
from nltk.util import ngrams

##  Часть 3. Нейросетевая 

### Нейронная сеть [4 балла]


Используйте  реккурентную нейронную сеть с  LSTM для решения задачи. В ней может быть несколько слоев с LSTM, несколько слоев c Bidirectional(LSTM).  У нейронной сети один выход, определяющий класс имени. 

Представление имени для классификации в этом случае: бинарная матрица размера (количество букв в алфавите $\times$ максимальная длина имени). Обозначим его через $x$. Если первая буква имени a, то $x[1][1] = 1$, если вторая – b, то  $x[2][1] = 1$ – то есть, используется one hot encoding.  

Не забудьте про регуляризацию нейронной сети дропаутами. 

Сравните результаты классификации разными методами. Какой метод лучше и почему?

Сравните результаты, получаемые при разных значениях дропаута, разных числах узлов на слоях нейронной сети по $F$-мере и аккуратности. В каких случаях нейронная сеть ошибается?

Если совсем не получается запрограммировать нейронную сеть самостоятельно, обратитесь к туториалу тут: https://github.com/divamgupta/lstm-gender-predictor

#### делаем one-hot-encoded данные

In [38]:
alphabet = set(''.join(df['name']))

In [20]:
maxlen = max(df['name'].map(lambda x: len(x)))
maxlen

15

In [46]:
def one_hot(stri, columns, rows):
    res = pd.DataFrame({col: [0]*rows for col in columns})
    for l in range(len(stri)):
        res[stri[l]][l] = 1
    return res

In [48]:
one_hot('ABC', alphabet, maxlen)

,,',-,A,B,C,D,E,F,G,...,q,r,s,t,u,v,w,x,y,z
0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [53]:
# X_train = df['name'].map(lambda x: one_hot(x, alphabet, maxlen))
# y_train = df['class']
X_train[1]

,,',-,A,B,C,D,E,F,G,...,q,r,s,t,u,v,w,x,y,z
0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
from sklearn.utils import shuffle

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import LSTM, TimeDistributed, Bidirectional
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, Flatten, Dropout

Using TensorFlow backend.


In [23]:
model = Sequential()
# бинарная матрица размера (количество букв в алфавите × максимальная длина имени)
model.add(Embedding(len(alphabet)+2, 30, input_length=maxlen))
model.add(LSTM(128, return_sequences = True))
model.add(Dropout(0.5))
model.add(LSTM(128, return_sequences = True))
# model.add(TimeDistributed(Dense(len(alphabet)+2, activation = 'softmax'))) # может быть потом я пойму зачем это
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [25]:
for iteration in range(1, 100):
    X_train_shuffled, y_train_shuffled = shuffle(x_train, y_train)
    model.fit(X_train_shuffled, y_train_shuffled, batch_size=len(x_train), epochs=1, verbose=0)

AttributeError: 'str' object has no attribute 'ndim'

In [ ]:
preds = model.predict(X_test, verbose=0)[0]

In [ ]:
from sklearn.metrics import classification_report
classification_report(y_test, preds)

In [ ]:
# другие методы
model = Sequential()
model.add(Embedding(len(alphabet)+2, 30, input_length=maxlen))
model.add(LSTM(128, return_sequences = True))
model.add(Dropout(0.5))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

for iteration in range(1, 100):
    X_train_shuffled, y_train_shuffled = shuffle(X_train,y_train)
    model.fit(X_train_shuffled, y_train_shuffled, batch_size=len(X_train), epochs=1, verbose = 0)
    
preds = model.predict(X_test, verbose=0)[0]

from sklearn.metrics import classification_report
classification_report(y_test, preds)

In [ ]:
model = Sequential()
model.add(Embedding(len(alphabet)+2, 30, input_length=maxlen))
model.add(LSTM(128, return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(128, return_sequences = True))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

for iteration in range(1, 100):
    X_train_shuffled, y_train_shuffled = shuffle(X_train,y_train)
    model.fit(X_train_shuffled, y_train_shuffled, batch_size=len(X_train), epochs=1, verbose = 0)
    
preds = model.predict(X_test, verbose=0)[0]

from sklearn.metrics import classification_report
classification_report(y_test, preds)

##  Часть 4. Сомнительная 

### Визуализация [1 балл]

Побочным продуктом обучения нейронной сети будут эмбеддинги символов – 26 векторов для всех букв латинского алфавита. Попробуйте использовать разные методы снижения размерности – SVD или TSNE – и изобразить точки, соответствующие разным символам, на плоскости. Получаются ли осмысленные и интерпретируемые кластеры? 